In [1]:
import os
import re
import numpy as np
import pandas as pd
from random import randint
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

In [2]:
#loading prediction file

data_name = "gsm8k"
# data_name = "mmlu"

path = "../data/routing_classifier_predictions"

if data_name == "gsm8k":
    filename = "gsm8k_cls_predictions.csv"
elif data_name == "mmlu":
    filename = "mmlu_cls_predictions.csv"
else:
    raise Exception(f"Dataset -- {data_name} not found.")

data_all= pd.read_csv(os.path.join(path,filename), encoding="utf-8")
data = data_all.loc[data_all['split'] == 'test']
print(data_all.shape, data.shape, data_name)
data_all.head()

(8792, 6) (1319, 6) gsm8k


,question,split,true_label,prediction,raw_predictions,confidence
0,Natalia sold clips to 48 of her friends in Apr...,train,"['maj_gemma-7b-it_CHAT', 'maj_gemma-7b_LM', 'm...","['maj_gemma-7b-it_CHAT', 'maj_gemma-7b_LM', 'm...",[1 1 1 1 1 1],[0.3574865 3.1160297 1.3018662 3.9527552 1.378...
1,Weng earns $12 an hour for babysitting. Yester...,train,"['maj_gemma-7b_LM', 'maj_metamath-7b_LM', 'maj...","['maj_gemma-7b-it_CHAT', 'maj_gemma-7b_LM', 'm...",[1 1 1 1 1 1],[1.1924486 3.2154894 1.7751658 3.3046634 1.896...
2,Betty is saving money for a new wallet which c...,train,"['maj_gemma-7b_LM', 'maj_llama2-13b-chat_CHAT'...","['maj_gemma-7b-it_CHAT', 'maj_gemma-7b_LM', 'm...",[1 1 1 1 1 1],[0.3417136 2.9823468 1.1348128 3.8447897 1.254...
3,"Julie is reading a 120-page book. Yesterday, s...",train,"['maj_gemma-7b-it_CHAT', 'maj_gemma-7b_LM', 'm...","['maj_gemma-7b_LM', 'maj_metamath-7b_LM', 'maj...",[0 1 0 1 0 1],[-1.2804217 1.9825261 -0.03160543 3.822210...
4,James writes a 3-page letter to 2 different fr...,train,['maj_metamath-7b_LM'],"['maj_gemma-7b_LM', 'maj_metamath-7b_LM', 'maj...",[0 1 0 1 0 1],[-1.8108426 1.4010394 -0.44269687 3.527691...


In [3]:
#change as per the need

if data_name == 'gsm8k':
    llms = eval(data_all['prediction'].values.tolist()[1])
    print(llms)
    print(data_all['raw_predictions'].values.tolist()[1])
    id2mod = {i:val for i, val in enumerate(llms)}
    mod2id = {val:i for i, val in enumerate(llms)}
    mod2lat = {'maj_gemma-7b-it_CHAT':0.70,  'maj_gemma-7b_LM':7.10, 'maj_llama2-13b-chat_CHAT':1.80, 'maj_metamath-7b_LM':4.70, 'maj_mistral-7b-inst_CHAT':1.00, 'maj_mistral-7b-lm_LM':3.70}
elif data_name == 'mmlu':
    llms = eval(data_all['prediction'].values.tolist()[11102])
    print(llms)
    print(data_all['raw_predictions'].values.tolist()[11102])
    id2mod = {i:val for i, val in enumerate(llms)}
    mod2id = {val:i for i, val in enumerate(llms)}
    mod2lat = {'maj_gemma-7b-it_CHAT':1.00,  "maj_llama2-7b-lm_LM":2.30, 'maj_gemma-7b_LM':3.00, 'maj_llama2-13b-chat_CHAT':4.80, 'maj_metamath-7b_LM':2.40, 'maj_mistral-7b-inst_CHAT':1.10, 'maj_mistral-7b-lm_LM':1.80}
    
print(id2mod)
print(mod2id)

['maj_gemma-7b-it_CHAT', 'maj_gemma-7b_LM', 'maj_llama2-13b-chat_CHAT', 'maj_metamath-7b_LM', 'maj_mistral-7b-inst_CHAT', 'maj_mistral-7b-lm_LM']
[1 1 1 1 1 1]
{0: 'maj_gemma-7b-it_CHAT', 1: 'maj_gemma-7b_LM', 2: 'maj_llama2-13b-chat_CHAT', 3: 'maj_metamath-7b_LM', 4: 'maj_mistral-7b-inst_CHAT', 5: 'maj_mistral-7b-lm_LM'}
{'maj_gemma-7b-it_CHAT': 0, 'maj_gemma-7b_LM': 1, 'maj_llama2-13b-chat_CHAT': 2, 'maj_metamath-7b_LM': 3, 'maj_mistral-7b-inst_CHAT': 4, 'maj_mistral-7b-lm_LM': 5}


In [4]:
# Collecting ground Truth, Predictions, and confidence scores
Global_List = []

def clean(labels, flag):
    if flag == 'conf':
        elabels = [list(map(float, re.findall(r'-?\d+(?:\.\d+)?(?:[eE][+\-]?\d+)?', s))) for s in labels]
    else:
        elabels = [list(map(int, item.strip('[]').split(' '))) for item in labels]    
    return elabels
        

truth = [ [1 if element in item else 0 for element in llms]  for item in data['true_label'].tolist()]
pred = [ [1 if element in item else 0 for element in llms]  for item in data['prediction'].tolist()]

pred_check =  clean(data['raw_predictions'].tolist(), 'pred')
raw_conf = clean(data['confidence'].tolist(), 'conf')

assert len(truth) == len(pred) == len(pred_check) == len(raw_conf)
for p, pc in zip(pred, pred_check):
    if p != pc:
        print(p,pc)

print("Length of the Test Set:", len(pred))
print("Ground Truth Sample:", truth[:10])
print("Predictions Sample:", pred[:10])
print("Confidence Sample:", raw_conf[:2])
print("Model Details:", llms)
Global_List.append(['Considered LLMs', llms, '---'])

Length of the Test Set: 1319
Ground Truth Sample: [[0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 0], [0, 0, 0, 1, 1, 0], [1, 1, 1, 0, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1]]
Predictions Sample: [[0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 1, 1], [0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0], [0, 1, 0, 1, 1, 0]]
Confidence Sample: [[-2.2803745, 0.19979207, -1.4978793, 2.4199407, -1.5842866, -0.6895504], [-0.386041, 2.6517174, 0.47759134, 4.136191, 1.0317405, 1.5089911]]
Model Details: ['maj_gemma-7b-it_CHAT', 'maj_gemma-7b_LM', 'maj_llama2-13b-chat_CHAT', 'maj_metamath-7b_LM', 'maj_mistral-7b-inst_CHAT', 'maj_mistral-7b-lm_LM']


In [5]:
# Assuming data_all is a pandas DataFrame with a column named "true_label"
# containing the multi-label data as strings
data_train = data_all.loc[data_all['split'] == 'train']
all_lab = [eval(item) for item in data_train["true_label"].tolist()]
# Flatten the label tensors into a single list
all_labels = [label for label_tensor in all_lab for label in label_tensor]

res = compute_class_weight(class_weight="balanced", classes=np.unique(all_labels), y=all_labels)
for n, v in zip(np.unique(all_labels), res):
    print(n, v)

maj_gemma-7b-it_CHAT 1.7108979483080202
maj_gemma-7b_LM 0.810425343935378
maj_llama2-13b-chat_CHAT 1.2322011130301285
maj_metamath-7b_LM 0.6479967706125744
maj_mistral-7b-inst_CHAT 1.193716304145752
maj_mistral-7b-lm_LM 1.0110218863171154


In [6]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(truth, pred, target_names=llms))
Global_List.append(['Classifier-f1-weighted', f1_score(truth, pred, average='weighted'), '000'])

                          precision    recall  f1-score   support

    maj_gemma-7b-it_CHAT       0.56      0.46      0.51       486
         maj_gemma-7b_LM       0.76      0.86      0.81       938
maj_llama2-13b-chat_CHAT       0.61      0.62      0.62       616
      maj_metamath-7b_LM       0.68      1.00      0.81       891
maj_mistral-7b-inst_CHAT       0.64      0.61      0.63       665
    maj_mistral-7b-lm_LM       0.73      0.72      0.73       788

               micro avg       0.68      0.75      0.71      4384
               macro avg       0.66      0.71      0.68      4384
            weighted avg       0.68      0.75      0.71      4384
             samples avg       0.63      0.63      0.58      4384



/Users/kvaditya/miniconda3/envs/its_v1/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kvaditya/miniconda3/envs/its_v1/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
import numpy as np
import math

def softmax_manual(scores):
    exp_scores = np.exp(scores - np.max(scores))
    return exp_scores / exp_scores.sum(axis=0)

def sigmoid(scores):
    return [(1 / (1 + math.exp(-x))) for x in scores]


conf = [sigmoid(item) for item in raw_conf]
print("Confidence Sample:", conf[:5])

Confidence Sample: [[0.09276143160787238, 0.5497825306584028, 0.18274203173045825, 0.9183352974108966, 0.1701892513538227, 0.3341330964687599], [0.404670711534072, 0.9341167626651471, 0.6171789437030931, 0.9842678395313634, 0.7372531888936387, 0.8189116397889744], [0.17980583862465088, 0.22932150823237127, 0.20091072785399008, 0.411763544207105, 0.20542057281884493, 0.20359967368149376], [0.6837884209103208, 0.9613524384272092, 0.8449381421616206, 0.9742395186807706, 0.8485207387632288, 0.9315968002616676], [0.0732129976165776, 0.5377422250936039, 0.18734527836434933, 0.9193480281483987, 0.16308518765599958, 0.3142760719112152]]


In [8]:
from collections import Counter

def lab_dist(all_in):
    temp = [ [ id2mod[i] for i, val in enumerate(item) if val !=0 ] for item in all_in]
    flattened_list = [item for sublist in temp for item in sublist]

    # Count the occurrences of each element
    element_counts = Counter(flattened_list)

    # Display the counts
    for element, count in element_counts.items():
        percentage = (count / len(all_in)) * 100
        print(f"{element}: {count} times ({percentage:.2f}%)")

print("*********** Ground Truth Label Distribution *************")
lab_dist(truth)
print("")
print("*********** Prediction Label Distribution ****************")
lab_dist(pred)

*********** Ground Truth Label Distribution *************
maj_gemma-7b_LM: 938 times (71.11%)
maj_llama2-13b-chat_CHAT: 616 times (46.70%)
maj_metamath-7b_LM: 891 times (67.55%)
maj_mistral-7b-inst_CHAT: 665 times (50.42%)
maj_mistral-7b-lm_LM: 788 times (59.74%)
maj_gemma-7b-it_CHAT: 486 times (36.85%)

*********** Prediction Label Distribution ****************
maj_gemma-7b_LM: 1067 times (80.89%)
maj_metamath-7b_LM: 1314 times (99.62%)
maj_llama2-13b-chat_CHAT: 624 times (47.31%)
maj_mistral-7b-inst_CHAT: 634 times (48.07%)
maj_mistral-7b-lm_LM: 778 times (58.98%)
maj_gemma-7b-it_CHAT: 398 times (30.17%)


In [9]:
print("********** Running Orecle Model *********")
# Oracle Model: Always selct best model
def oreacle(all_lab):
    acc_count = 0
    temp_count = 0
    llm_count = len(llms)*[0]
    for lab in all_lab:
        for i, val in enumerate(lab):
            if val == 1:
                acc_count += 1
                llm_count[i] += 1
                break
            if i ==len(llms)-1:
                temp_count += 1 
   
    assert temp_count + sum(llm_count) == len(all_lab)
    return acc_count, llm_count
        

acc, llm_count_list = oreacle(truth)
facc = acc/(len(truth))
lat = sum([ item* mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
flat=lat/sum(llm_count_list)
print("Max Accuracy (%) : ", facc*100)
print("Min Latency (Sec/Prompt) : ", flat)

Global_List.append(['Oracle', facc*100, flat])

********** Running Orecle Model *********
Max Accuracy (%) :  87.18726307808946
Min Latency (Sec/Prompt) :  3.859391304347826


In [10]:
print("***********Running Random Model ************")

def random(all_lab):
    acc_count = 0
    temp_count = 0
    llm_count = len(llms)*[0]
    for lab in all_lab:
        random_index = randint(0, len(lab) - 1)
        if lab[random_index] >= 0.8:
            acc_count += 1
            llm_count[random_index] += 1
        else:
            temp_count += 1 
            
    assert temp_count + sum(llm_count) == len(all_lab)
    return acc_count, llm_count
        
acc_list, lat_list = [],[]
for i in range(1000): 
    acc, llm_count_list = random(truth)
    facc = acc/(len(truth))
    lat = sum([ item*mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
    flat=lat/sum(llm_count_list)
    acc_list.append(facc)
    lat_list.append(flat)

print("Max Accuracy (100) : ", sum(acc_list)/len(acc_list)*100)
print("Latency (Sec/Prompt) : ", sum(lat_list)/len(lat_list))

Global_List.append(['Random', sum(acc_list)/len(acc_list)*100,  sum(lat_list)/len(lat_list)])

***********Running Random Model ************
Max Accuracy (100) :  55.37801364670209
Latency (Sec/Prompt) :  3.6233322536554033


In [11]:
print("********** Running Different Baseline Models *********")
# Oracle Model: Always selct best model
def baseline(all_lab):
    acc_count = 0
    temp_count = 0
    llm_count = len(llms)*[0]
    for lab in all_lab:
        for i, val in enumerate(lab):
            if val == 1:
                acc_count += 1
                llm_count[i] += 1
                break
            if i ==len(llms)-1:
                temp_count += 1 
   
    assert temp_count + sum(llm_count) == len(all_lab)
    return acc_count, llm_count
        
def all_baselines(filtered_truth):
    acc, llm_count_list = baseline(filtered_truth)
    facc = acc/(len(truth))
    lat = sum([ item*mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
    flat=lat/sum(llm_count_list)
    return facc, flat


from itertools import permutations, chain, combinations
import random

def generate_arrangements(values):
    # Generate all permutations of lists of integers in the range (0, values)
    arrangements = list(permutations(range(values)))

    # Extract unique individual numbers
    unique_numbers = set(chain.from_iterable(arrangements))
    # Generate all combinations of individual numbers
    individual_lists = [[num] for num in unique_numbers]

    # Combine individual numbers to form arrangements
    combined_arrangements = []
    for num in list(unique_numbers):
        if num != list(unique_numbers)[0] and num != list(unique_numbers)[-1]:
            combined_arrangements += [list(comb) for comb in combinations(unique_numbers, num) ]

    all_elements_list = list(range(values))
    combined_arrangements.append(all_elements_list)

    # Limit to 10 arrangements
    flist = combined_arrangements[:values]
    llist = combined_arrangements[-1]
    mlist_temp = combined_arrangements[values:][:-1]
    mlist = random.sample(mlist_temp, min(len(mlist_temp), 10))
    filtered_arrangements = flist+mlist+[llist]
    sorted_nested_list = sorted(filtered_arrangements, key=len)
    return sorted_nested_list

baseline_config = generate_arrangements(len(llms))
if len(llms) == 2:
    baseline_config.append([0])
    baseline_config.append([1])
print("Configurations:", baseline_config)
    
for base in baseline_config:
    filt_truth = [[val if i in base else 0 for i, val in enumerate(item)] for item in truth]
    out_acc, out_lat = all_baselines(filt_truth)
    Global_List.append([[ id2mod[ind] for ind in base], out_acc*100, out_lat])
    print("Current Model is/are:",  [ id2mod[ind] for ind in base])
    print("Max Accuracy (%) : ", out_acc*100)
    print("Min Latency (Sec/Prompt) : ", out_lat)

********** Running Different Baseline Models *********
Configurations: [[0], [1], [2], [3], [4], [5], [0, 3], [1, 3], [2, 4], [1, 3, 4], [0, 1, 4], [0, 4, 5], [0, 2, 5], [1, 2, 3, 4], [0, 1, 2, 5], [0, 2, 4, 5], [0, 1, 2, 3, 4, 5]]
Current Model is/are: ['maj_gemma-7b-it_CHAT']
Max Accuracy (%) :  36.84609552691433
Min Latency (Sec/Prompt) :  0.7
Current Model is/are: ['maj_gemma-7b_LM']
Max Accuracy (%) :  71.1144806671721
Min Latency (Sec/Prompt) :  7.1
Current Model is/are: ['maj_llama2-13b-chat_CHAT']
Max Accuracy (%) :  46.70204700530705
Min Latency (Sec/Prompt) :  1.7999999999999998
Current Model is/are: ['maj_metamath-7b_LM']
Max Accuracy (%) :  67.55117513267626
Min Latency (Sec/Prompt) :  4.7
Current Model is/are: ['maj_mistral-7b-inst_CHAT']
Max Accuracy (%) :  50.41698256254739
Min Latency (Sec/Prompt) :  1.0
Current Model is/are: ['maj_mistral-7b-lm_LM']
Max Accuracy (%) :  59.74222896133434
Min Latency (Sec/Prompt) :  3.7000000000000006
Current Model is/are: ['maj_gemma-7b

In [12]:
print("******* Upper Bound of Performance with Optimal classifer and Policy******")
# this will be calculated with with function as = f(prediction, ground truth)

def uperbound(truth_list, pred_list):
    acc_count, temp_count = 0, 0
    llm_count = len(llms)*[0]
    for i, (t, p) in enumerate(zip(truth_list, pred_list)):
        for j, pval in enumerate(p):
            if t[j] ==p[j] and pval == 1:
                acc_count += 1 
                llm_count[j] += 1
                break
            if j==len(llms)-1:
                temp_count += 1      
    assert temp_count + sum(llm_count) == len(truth_list)
    return acc_count, llm_count
        

acc, llm_count_list = uperbound(truth, pred)
facc = acc/(len(truth))
lat = sum([ item*mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
flat=lat/sum(llm_count_list)
print("Max Accuracy (%) : ", facc*100)
print("Min Latency (Sec/Prompt) : ", flat)

Global_List.append(['Upperbound Performance (Classifier)', facc*100, flat])

******* Upper Bound of Performance with Optimal classifer and Policy******
Max Accuracy (%) :  79.68157695223654
Min Latency (Sec/Prompt) :  5.16022835394862


In [13]:
print("******* Proposed Policies: Policy 1: ArgMax ******")
# this will be calculated with with function as = f(prediction, ground truth, confidence)

def policy1(pred_list, conf_list, truth_list):
    acc_count, temp_count = 0, 0
    llm_count = len(llms)*[0]
    for i, (p, c, t) in enumerate(zip(pred_list, conf_list, truth_list)):
        try: 
            maxconf_ind = c.index(max(c))
            if p[maxconf_ind] == 1 and t[maxconf_ind] == 1:
                acc_count += 1 
                llm_count[maxconf_ind] += 1
            else:
                temp_count += 1   
        except:
            print(c, maxconf_ind, p, t)
            raise Exception()
    
    assert temp_count + sum(llm_count) == len(pred_list)
    return acc_count, llm_count
        

acc, llm_count_list = policy1(pred, conf, truth)
facc = acc/(len(pred))
lat = sum([ item*mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
flat=lat/sum(llm_count_list)
print("Accuracy (%): ", facc*100)
print("Latency (Sec/Prompt) : ", flat)

Global_List.append(['Policy1 (ConfArgMax) :', facc*100, flat])

******* Proposed Policies: Policy 1: ArgMax ******
Accuracy (%):  67.62699014404852
Latency (Sec/Prompt) :  4.767264573991031


In [14]:
import random

print("******* Proposed Policies (Policy2) : random pic from top k confidance scores if confidence is less then X value ******")
# this will be calculated with with function as = f(prediction, ground truth, confidence)

import heapq
def top_index(my_list):
    indices_of_highest = [my_list.index(x) for x in heapq.nlargest(3, my_list)]
    return indices_of_highest

def policy1(pred_list, conf_list, truth_list):
    acc_count, temp_count = 0, 0
    llm_count = len(llms)*[0]
    for i, (p, c, t) in enumerate(zip(pred_list, conf_list, truth_list)):
        maxconf_ind = c.index(max(c))
        if c[maxconf_ind] <= 0.60:
            rand_indx = random.choice(top_index(c))
            if p[rand_indx] == 1 and t[rand_indx] == 1:
                acc_count += 1 
                llm_count[rand_indx] += 1
            else:
                temp_count += 1
        else:
            maxconf_ind = c.index(max(c))
            if p[maxconf_ind] == 1 and t[maxconf_ind] == 1:
                acc_count += 1 
                llm_count[maxconf_ind] += 1
            else:
                temp_count += 1   
    
    assert temp_count + sum(llm_count) == len(pred_list)
    return acc_count, llm_count
        

acc, llm_count_list = policy1(pred, conf, truth)
facc = acc/(len(pred))
lat = sum([ item*mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
flat=lat/sum(llm_count_list)
print("Accuracy (%): ", facc*100)
print("Latency (Sec/Prompt) : ", flat)

Global_List.append(['Policy2 (Select Random) :', facc*100, flat])

******* Proposed Policies (Policy2) : random pic from top k confidance scores if confidence is less then X value ******
Accuracy (%):  67.47536012130402
Latency (Sec/Prompt) :  4.7674157303370785


In [15]:
print("******* Proposed Policies: Policy 3: Predict output ******")
# this will be calculated with with function as = f(prediction, ground truth, confidence)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from collections import Counter

def collect_input_target(data_temp):
    temp_conf = clean(data_temp['confidence'].tolist(), 'conf')
    input_conf = [ sigmoid(item) for item in temp_conf]
    target_truth = [ [1 if element in item else 0 for element in llms]  for item in data['true_label'].tolist()]
    target_conf = [ conf[item.index(1)] if 1 in item else 0.5 for item, conf in zip(target_truth, input_conf) ]
    
    count = 0
    x,y =[], []
    for ix, iy in zip(input_conf, target_conf):
        if len(ix) == len(llms) and len([iy]) == 1:
            x.append(ix)
            y.append(iy)
        else:
            count= count +1
    print("Incorect Rows:", count)
    return x, y


data_train = data_all.loc[data_all['split'] == 'train']
data_val = data_all.loc[data_all['split'] == 'val']

train_x, train_y = collect_input_target(data_train)
val_x, val_y = collect_input_target(data_val)


model = LinearRegression()
model.fit(train_x, train_y)

y_pred = model.predict(val_x)
mse = mean_squared_error(val_y, y_pred)
print(f'Mean Squared Error on Dev Set: {mse}')

predicted_output = model.predict(conf).tolist()

predicted_policy_index =[]
for con, pred in zip(conf, predicted_output):
    temp_list = [ item-pred for item in con]
    argmax = temp_list.index(max(temp_list))
    predicted_policy_index.append(argmax)
assert len(predicted_policy_index) == len(conf)

def policy3(truth_list, pred_ind):
    acc_count  = 0    
    llm_count = len(llms)*[0]
    for item, val in zip(truth_list, pred_ind):
        if item[val] == 1:
            acc_count += 1
            llm_count[val] += 1
    return acc_count, llm_count
        

acc, llm_count_list = policy3(truth, predicted_policy_index)
facc = acc/(len(truth))
lat = sum([ item*mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
flat=lat/sum(llm_count_list)
print("Accuracy (%): ", facc*100)
print("Latency (Sec/Prompt) : ", flat)

Global_List.append(['Policy3 (Prediction) :',facc*100, flat])

******* Proposed Policies: Policy 3: Predict output ******
Incorect Rows: 0
Incorect Rows: 0
Mean Squared Error on Dev Set: 0.05219229185139534
Accuracy (%):  67.70280515542078
Latency (Sec/Prompt) :  4.767189249720045


In [16]:
print("******* Proposed Policies: Policy 4: Sorted Prediction output ******")
# this will be calculated with with function as = f(prediction, ground truth, confidence)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from collections import Counter

# obtain new mapping based on frequency
target_truth = [ [1 if element in item else 0 for element in llms]  for item in data['true_label'].tolist()]
temp = [ [ id2mod[i] for i, val in enumerate(item) if val !=0 ] for item in target_truth]
flattened_list = [item for sublist in temp for item in sublist]
element_counts = Counter(flattened_list)

sorted_dict = {item:i for i, (item, count) in enumerate(sorted(element_counts.items(), key=lambda x: x[1]))}
print(mod2id)
print(sorted_dict)
map_dict = {}
for key, value in sorted_dict.items():
    map_dict[sorted_dict[key]] = mod2id[key]
print(map_dict)
    
def collect_input_target(data_temp):
    temp_conf = clean(data_temp['confidence'].tolist(), 'conf')
    input_conf = [ sigmoid(item) for item in temp_conf]
    new_input_conf = [  [ item[map_dict[i]] for i, val in enumerate(item)] for item in input_conf]
    
    target_truth = [ [1 if element in item else 0 for element in llms]  for item in data['true_label'].tolist()]
    new_target_truth = [  [ item[map_dict[i]] for i, val in enumerate(item)] for item in target_truth]
    target_conf = [ conf[item.index(1)] if 1 in item else 0.5 for item, conf in zip(new_target_truth, new_input_conf) ]
    
    count = 0
    x,y =[], []
    for ix, iy in zip(input_conf, target_conf):
        if len(ix) == len(llms) and len([iy]) == 1:
            x.append(ix)
            y.append(iy)
        else:
            count= count +1
    print("Incorect Rows:", count)
    return x, y


data_train = data_all.loc[data_all['split'] == 'train']
data_val = data_all.loc[data_all['split'] == 'val']

train_x, train_y = collect_input_target(data_train)
val_x, val_y = collect_input_target(data_val)


model = LinearRegression()
model.fit(train_x, train_y)

y_pred = model.predict(val_x)
mse = mean_squared_error(val_y, y_pred)
print(f'Mean Squared Error on Dev Set: {mse}')

predicted_output = model.predict(conf).tolist()

predicted_policy_index =[]
for con, pred in zip(conf, predicted_output):
    temp_list = [ item-pred for item in con]
    argmax = temp_list.index(max(temp_list))
    predicted_policy_index.append(argmax)
assert len(predicted_policy_index) == len(conf)

def policy4(truth_list, pred_ind):
    acc_count  = 0    
    llm_count = len(llms)*[0]
    for item, val in zip(truth_list, pred_ind):
        if item[val] == 1:
            acc_count += 1
            llm_count[val] += 1
    return acc_count, llm_count
        

new_truth = [  [ item[map_dict[i]] for i, val in enumerate(item)] for item in truth]
acc, llm_count_list = policy4(new_truth, predicted_policy_index)
facc = acc/(len(new_truth))
lat = sum([ item*mod2lat[id2mod[i]] for i, item in enumerate(llm_count_list)])
flat=lat/sum(llm_count_list)
print("Accuracy (%): ", facc*100)
print("Latency (Sec/Prompt) : ", flat)

Global_List.append(['Policy4 (Sorted Prediction) :', facc*100, flat])

******* Proposed Policies: Policy 4: Sorted Prediction output ******
{'maj_gemma-7b-it_CHAT': 0, 'maj_gemma-7b_LM': 1, 'maj_llama2-13b-chat_CHAT': 2, 'maj_metamath-7b_LM': 3, 'maj_mistral-7b-inst_CHAT': 4, 'maj_mistral-7b-lm_LM': 5}
{'maj_gemma-7b-it_CHAT': 0, 'maj_llama2-13b-chat_CHAT': 1, 'maj_mistral-7b-inst_CHAT': 2, 'maj_mistral-7b-lm_LM': 3, 'maj_metamath-7b_LM': 4, 'maj_gemma-7b_LM': 5}
{0: 0, 1: 2, 2: 4, 3: 5, 4: 3, 5: 1}
Incorect Rows: 0
Incorect Rows: 0
Mean Squared Error on Dev Set: 0.03729692368496198
Accuracy (%):  59.59059893858984
Latency (Sec/Prompt) :  4.7732824427480915


In [17]:
global_df = pd.DataFrame(Global_List, columns=["Model", "Accuracy (%)", "Latency (Sec/prompt)"])
global_df

,Model,Accuracy (%),Latency (Sec/prompt)
0,Considered LLMs,"[maj_gemma-7b-it_CHAT, maj_gemma-7b_LM, maj_ll...",---
1,Classifier-f1-weighted,0.705367,000
2,Oracle,87.187263,3.859391
3,Random,55.378014,3.623332
4,[maj_gemma-7b-it_CHAT],36.846096,0.7
5,[maj_gemma-7b_LM],71.114481,7.1
6,[maj_llama2-13b-chat_CHAT],46.702047,1.8
7,[maj_metamath-7b_LM],67.551175,4.7
8,[maj_mistral-7b-inst_CHAT],50.416983,1.0
9,[maj_mistral-7b-lm_LM],59.742229,3.7


In [18]:
global_df.to_csv(f'./routing_results_{data_name}.csv', index=False)